In [1]:
%load_ext autoreload
%autoreload 2
import os
import sys
from IPython.display import IFrame

module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

from generator import RoadNetwork
import osmnx as ox
import pandas as pd
import geopandas as gpd
import numpy as np
import fmm
from shapely.geometry import LineString
import swifter

In [2]:
network = RoadNetwork("Porto, Portugal", network_type="drive", retain_all=True, truncate_by_edge=True)

In [15]:
import folium
gdf_nodes, gdf_edges = ox.graph_to_gdfs(network.G)
bounds = gdf_nodes.geometry.total_bounds
print(bounds)
m = folium.Map(
    location=[41.1372482, -8.689151],
    zoom_start=11,
)

folium.Rectangle(bounds=[(bounds[1], bounds[0]),(bounds[3], bounds[2])], color='#ff7800', fill=True, fill_color='#ffff00', fill_opacity=0.2).add_to(m)
m

[-8.6886988 41.1402264 -8.5559592 41.1858236]


In [19]:
from shapely.geometry import box

bbox = box(*bounds)
poly_gdf = gpd.GeoDataFrame([1], geometry=[bbox], crs="EPSG:4326")
poly_gdf

,0,geometry
0,1,"POLYGON ((-8.55596 41.14023, -8.55596 41.18582..."


In [4]:
from shapely import wkt
df = pd.read_csv("../datasets/trajectories/Porto/mapped.csv")
df['POLYLINE'] = df['POLYLINE'].swifter.apply(wkt.loads)
gdf = gpd.GeoDataFrame(df, crs='epsg:4326', geometry="POLYLINE")

<class 'pandas.core.series.Series'>


In [21]:
gdf_clipped = gdf.clip(poly_gdf, keep_geom_type=True)

,Unnamed: 0,TRIP_ID,CALL_TYPE,ORIGIN_CALL,ORIGIN_STAND,TAXI_ID,TIMESTAMP,DAY_TYPE,MISSING_DATA,POLYLINE
1609155,1677474,1403587051620000506,A,23057.0,NaN,20000506,1403587051,A,False,"LINESTRING (-8.59265 41.15743, -8.59244 41.157..."
1443076,1504160,1400672773620000198,A,30643.0,NaN,20000198,1400672773,A,False,"LINESTRING (-8.57206 41.16549, -8.57205 41.165..."
1095919,1141232,1394024019620000672,A,30643.0,NaN,20000672,1394024019,A,False,"LINESTRING (-8.57203 41.16550, -8.57204 41.165..."
1521689,1586365,1402054320620000408,A,30643.0,NaN,20000408,1402054320,A,False,"LINESTRING (-8.57203 41.16542, -8.57197 41.164..."
1280010,1334213,1397723369620000118,A,39404.0,NaN,20000118,1397723369,A,False,"MULTILINESTRING ((-8.56815 41.16768, -8.56732 ..."
...,...,...,...,...,...,...,...,...,...,...
769044,797921,1387448158620000246,A,6084.0,NaN,20000246,1387448158,A,False,"MULTILINESTRING ((-8.65266 41.16687, -8.65261 ..."
746895,774463,1387001117620000360,C,NaN,NaN,20000360,1387001117,A,False,"MULTILINESTRING ((-8.60626 41.14686, -8.60659 ..."
104275,107886,1374591610620000177,B,NaN,35.0,20000177,1374591610,A,False,"LINESTRING (-8.64958 41.16733, -8.64960 41.167..."
305280,316090,1378885846620000251,B,NaN,16.0,20000251,1378885846,A,False,"LINESTRING (-8.62762 41.17508, -8.62682 41.173..."


In [10]:
network.save(path="../osm_data/porto")


/Users/ironmonkeyapps/Desktop/Studium/Master/Semester4/Abschlussarbeit/Road-Network-Embedding-Generator/generator/road_network.py:46: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  gdf_nodes.to_file(path + "/nodes.shp", encoding="utf-8")


In [2]:
td = pd.read_csv("../datasets/trajectories/Porto/mapped.csv")
tdtemp = td.loc[:, ["TRIP_ID", "POLYLINE"]]
tdtemp["id"] = np.arange(1, tdtemp.shape[0]+1)
tdtemp.to_csv("../datasets/trajectories/Porto/mapped_id_poly.csv", sep=";", index=False)

In [3]:
network.fmm_trajectorie_mapping()

In [5]:
G = network.G
gdf_nodes, gdf_edges = ox.graph_to_gdfs(
        G,
        nodes=True, edges=True,
        node_geometry=True,
        fill_edge_geometry=True)

gdf_edges.describe(include='all')

/Users/ironmonkeyapps/miniconda3/envs/road-emb/lib/python3.9/site-packages/pandas/core/dtypes/cast.py:122: ShapelyDeprecationWarning: The array interface is deprecated and will no longer work in Shapely 2.0. Convert the '.coords' to a numpy array instead.
  arr = construct_1d_object_array_from_listlike(values)


,osmid,oneway,highway,length,geometry,lanes,ref,maxspeed,bridge,name,width,access,junction,tunnel,area
count,11308.0,11308,11308,11308.000000,11308,2316,480,854,164,10203,205,223,198,74,6
unique,4974.0,2,19,NaN,11304,12,32,18,4,1910,9,5,2,3,1
top,12301715.0,False,residential,NaN,"LINESTRING (-8.6667028 41.1481788, -8.6668598 ...",2,EN 12,50,yes,Avenida de Fernão de Magalhães,4,yes,roundabout,yes,highway
freq,30.0,7400,5779,NaN,2,1268,344,632,128,102,170,182,193,47,6
mean,NaN,NaN,NaN,85.117299,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
std,NaN,NaN,NaN,94.203150,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
min,NaN,NaN,NaN,1.564000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25%,NaN,NaN,NaN,30.470000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
50%,NaN,NaN,NaN,60.193000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
75%,NaN,NaN,NaN,107.726500,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [6]:
m1 = ox.plot_graph_folium(G, popup_attribute="name", weight=2, color="#FFFFFF", tiles="cartodbdark_matter")
filepath = "graph.html"
m1.save(filepath)
#IFrame(filepath, width=600, height=500)

In [4]:
# load porto trajectory dataset
td = pd.read_csv("../datasets/trajectories/Porto/train.csv")

In [5]:
td = td[td["MISSING_DATA"]==False]
sample = np.matrix(td.loc[0, "POLYLINE"]).reshape(-1, 2)
gtd = gpd.GeoDataFrame(data=sample, index=np.arange(1, len(sample)+1), columns=["LONG", "LAT"], crs="EPSG:4326", geometry=gpd.points_from_xy(sample[:, 0], sample[:, 1]))
print(gtd.to_wkt())
#network.map_trajectorie(coordinates=gtd)

        LONG        LAT                     geometry
1  -8.618643  41.141412  POINT (-8.618643 41.141412)
2  -8.618499  41.141376  POINT (-8.618499 41.141376)
3  -8.620326  41.142510   POINT (-8.620326 41.14251)
4  -8.622153  41.143815  POINT (-8.622153 41.143815)
5  -8.623953  41.144373  POINT (-8.623953 41.144373)
6  -8.626680  41.144778   POINT (-8.62668 41.144778)
7  -8.627373  41.144697  POINT (-8.627373 41.144697)
8  -8.630226  41.145210   POINT (-8.630226 41.14521)
9  -8.632746  41.146920   POINT (-8.632746 41.14692)
10 -8.631738  41.148225  POINT (-8.631738 41.148225)
11 -8.629938  41.150385  POINT (-8.629938 41.150385)
12 -8.629110  41.151213   POINT (-8.62911 41.151213)
13 -8.629128  41.151240   POINT (-8.629128 41.15124)
14 -8.628786  41.152203  POINT (-8.628786 41.152203)
15 -8.628687  41.152374  POINT (-8.628687 41.152374)
16 -8.628759  41.152518  POINT (-8.628759 41.152518)
17 -8.630838  41.152680   POINT (-8.630838 41.15268)
18 -8.632323  41.153022  POINT (-8.632323 41.1